In [3]:
#random matrix generator
#-----------------------------------------------------------------------------------------------------------------------------
import cv2
import numpy as np
import random

# Create a blank image (white background)
height, width = 502, 502
cell_size = 20
rows, cols = height // cell_size, width // cell_size
maze = np.ones((height, width, 3), dtype=np.uint8) * 255

# Draw grid lines (walls)
for i in range(rows + 1):
    cv2.line(maze, (0, i * cell_size), (width, i * cell_size), (0, 0, 0), 2)
for j in range(cols + 1):
    cv2.line(maze, (j * cell_size, 0), (j * cell_size, height), (0, 0, 0), 2)


visited = np.zeros((rows, cols), dtype=bool)

def carve_path(r, c):
    visited[r, c] = True
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    random.shuffle(directions)
    
    for dr, dc in directions:
        nr, nc = r + dr, c + dc
        if 0 <= nr < rows and 0 <= nc < cols and not visited[nr, nc]:
            # Remove wall between cells
            if dr == 0 and dc == 1:  # Right
                cv2.line(maze, ((c+1)*cell_size, r*cell_size+2), ((c+1)*cell_size, (r+1)*cell_size-2), (255, 255, 255), 2)
            elif dr == 1 and dc == 0:  # Down
                cv2.line(maze, (c*cell_size+2, (r+1)*cell_size), ((c+1)*cell_size-2, (r+1)*cell_size), (255, 255, 255), 2)
            elif dr == 0 and dc == -1:  # Left
                cv2.line(maze, (c*cell_size, r*cell_size+2), (c*cell_size, (r+1)*cell_size-2), (255, 255, 255), 2)
            elif dr == -1 and dc == 0:  # Up
                cv2.line(maze, (c*cell_size+2, r*cell_size), ((c+1)*cell_size-2, r*cell_size), (255, 255, 255), 2)
            
            carve_path(nr, nc)

# Start from top-left corner
carve_path(0, 0)

cv2.imshow('Maze', maze)
cv2.imwrite('generated_maze.png', maze)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [4]:
import cv2
import numpy as np
import os

# Define cell size (adjust based on your maze image)
CELL_SIZE = 20
np.set_printoptions(suppress=True, formatter={'all':lambda x: str(int(x))})

def readImage(img_file_path):
    binary_img = None
    img = cv2.imread(img_file_path, 0)
    ret, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
    binary_img = img
    return binary_img

def blockwork(img, coordinate):
    size = CELL_SIZE
    h = CELL_SIZE*(coordinate[0]+1)
    w = CELL_SIZE*(coordinate[1]+1)
    h0 = CELL_SIZE*coordinate[0]
    w0 = CELL_SIZE*coordinate[1]
    block = img[h0:h, w0:w]
#     cv2.imshow("block",block)
#     cv2.imshow("maze",img)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    up = bool(block[0, int(size/2)]) * 1000
    down = bool(block[int(size-1), int(size/2)]) * 100
    left = bool(block[int(size/2), 0]) * 10
    right = bool(block[int(size/2), int(size-1)]) * 1
    edge = up + down + left + right
    return edge,block




#20 X 20 matrix
# shape=(25,25)
# matrix=np.zeros(shape)
# for i in range(25):
#     for j in range(25):
#         matrix[i,j]=int(blockwork(img,[i,j])[0])
        
# print(matrix)


In [5]:
def solveMaze(original_binary_img, initial_point, final_point, no_cells_height, no_cells_width):
    # Create edge array for the maze
    edgearray = []
    img = original_binary_img
    for i in range(no_cells_height):
        edgearray.append([])
        for j in range(no_cells_width):
            sz = [i, j]
            edge, block = blockwork(img, sz)
            edgearray[i].append(edge)
    
    edge = edgearray
    shortestPath = []
    sp = []
    rec = [0]
    p = 0
    sp.append(list(initial_point))
    
    while True:
        h, w = sp[p][0], sp[p][1]
        # h stands for height and w stands for width
        
        if sp[-1] == list(final_point):
            break
            
        if edge[h][w] > 0:
            rec.append(len(sp))
            
        if edge[h][w] > 999:
            # If this edge is open upwards 
            edge[h][w] = edge[h][w] - 1000
            h = h - 1
            sp.append([h, w])
            edge[h][w] = edge[h][w] - 100
            p = p + 1
            continue
            
        if edge[h][w] > 99:
            # If the edge is open downward
            edge[h][w] = edge[h][w] - 100
            h = h + 1
            sp.append([h, w])
            edge[h][w] = edge[h][w] - 1000
            p = p + 1
            continue
            
        if edge[h][w] > 9:
            # If the edge is open left
            edge[h][w] = edge[h][w] - 10
            w = w - 1
            sp.append([h, w])
            edge[h][w] = edge[h][w] - 1
            p = p + 1
            continue
            
        if edge[h][w] == 1:
            # If the edge is open right
            edge[h][w] = edge[h][w] - 1
            w = w + 1
            sp.append([h, w])
            edge[h][w] = edge[h][w] - 10
            p = p + 1
            continue
            
        else:
            # Removing the coordinates that are closed or don't show any path   
            sp.pop()
            rec.pop()
            p = rec[-1]
    
    for i in sp:
        shortestPath.append(tuple(i))
        
    return shortestPath


In [6]:
img=readImage("./generated_maze.png")
ans= solveMaze(img,[0,0],[24,24],25,25)
print(ans)

[(0, 0), (1, 0), (2, 0), (2, 1), (3, 1), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (7, 1), (8, 1), (8, 2), (7, 2), (6, 2), (6, 1), (5, 1), (4, 1), (4, 2), (4, 3), (3, 3), (3, 2), (2, 2), (1, 2), (1, 1), (0, 1), (0, 2), (0, 3), (0, 4), (1, 4), (1, 3), (2, 3), (2, 4), (2, 5), (1, 5), (1, 6), (2, 6), (3, 6), (3, 5), (4, 5), (4, 4), (5, 4), (5, 3), (6, 3), (7, 3), (8, 3), (9, 3), (9, 2), (9, 1), (9, 0), (10, 0), (10, 1), (11, 1), (11, 0), (12, 0), (12, 1), (13, 1), (13, 2), (13, 3), (12, 3), (12, 2), (11, 2), (10, 2), (10, 3), (11, 3), (11, 4), (12, 4), (13, 4), (13, 5), (14, 5), (14, 4), (14, 3), (14, 2), (15, 2), (15, 3), (15, 4), (16, 4), (16, 5), (15, 5), (15, 6), (15, 7), (15, 8), (16, 8), (17, 8), (18, 8), (18, 9), (17, 9), (16, 9), (15, 9), (14, 9), (14, 8), (14, 7), (14, 6), (13, 6), (13, 7), (12, 7), (12, 8), (11, 8), (11, 9), (10, 9), (10, 10), (9, 10), (8, 10), (8, 11), (8, 12), (8, 13), (7, 13), (7, 12), (6, 12), (5, 12), (5, 11), (6, 11), (6, 10), (7, 10), (7, 9), (6, 9), (5, 9)

In [7]:
def pathHighlight(img, ip, fp, path):
    size = CELL_SIZE
    
    # Check if image is grayscale and convert to color if needed
    if len(img.shape) == 2 or img.shape[2] == 1:
        path_img = cv2.cvtColor(img.copy(), cv2.COLOR_GRAY2BGR)
    else:
        path_img = img.copy()
    
    # Convert path coordinates to pixel coordinates
    pixel_path = [(CELL_SIZE * (coord[1] + 0.5), CELL_SIZE * (coord[0] + 0.5)) for coord in path]
    pixel_path = np.array(pixel_path, np.int32).reshape((-1, 1, 2))
    
    # Draw the path with a thicker line
    cv2.polylines(path_img, [pixel_path], False, (0, 0, 255), thickness=3)
    
    # Highlight start and end points with larger circles
    cv2.circle(path_img, (int(CELL_SIZE * (ip[1] + 0.5)), int(CELL_SIZE * (ip[0] + 0.5))), 7, (255, 0, 0), -1)
    cv2.circle(path_img, (int(CELL_SIZE * (fp[1] + 0.5)), int(CELL_SIZE * (fp[0] + 0.5))), 7, (0, 255, 0), -1)
    
    return path_img


In [8]:
path_img=pathHighlight(img,[0,0],[24,24],ans)


In [9]:
cv2.imshow("maze",img)
cv2.imshow("maze_solved",path_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
#real time path ----------------------------------------------------------------------

In [30]:
def drawPathRealtime(img, path, cell_size):
    # Create a copy of the original image
    if len(img.shape) == 2 or img.shape[2] == 1:
        path_img = cv2.cvtColor(img.copy(), cv2.COLOR_GRAY2BGR)
    else:
        path_img = img.copy()
    
    # Convert all path coordinates to pixel coordinates
    pixel_path = [(cell_size * (coord[1] + 0.5), cell_size * (coord[0] + 0.5)) for coord in path]
    
    # Display the starting image
    cv2.imshow('Maze Solving', path_img)
    cv2.waitKey(100)  # Short delay
    
    img_array=[]
    
    # Draw the path incrementally
    for i in range(1, len(pixel_path)):
        # Get current segment
        segment = np.array([pixel_path[i-1], pixel_path[i]], np.int32).reshape((-1, 1, 2))
        
        # Draw the segment
        cv2.polylines(path_img, [segment], False, (0, 0, 255), thickness=3)
        
        # Draw a circle at the current point for better visualization
        cv2.circle(path_img, (int(pixel_path[i][0]), int(pixel_path[i][1])), 3, (255, 0, 0), 0)
        
        # Display the updated image
        cv2.imshow('Maze Solving', path_img)
        img_array.append(path_img.copy())
        
        # Control the speed of animation
        key = cv2.waitKey(50)  # 50ms delay between steps
        if key == 27:  # ESC key to exit
            break
    
    # Final display with completed path
    cv2.circle(path_img, (int(pixel_path[0][0]), int(pixel_path[0][1])), 5, (0, 255, 0), -1)  # Start
    cv2.circle(path_img, (int(pixel_path[-1][0]), int(pixel_path[-1][1])), 5, (255, 0, 0), -1)  # End
    cv2.imshow('Maze Solving', path_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return img_array

# Example usage:
# drawPathRealtime(maze_img, shortest_path, CELL_SIZE)

In [31]:
import glob
img_array=drawPathRealtime(img,ans,20)
out = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 15, (502,502))

# Step 4: Write images to video
for i in range(len(img_array)):
    out.write(img_array[i])

# Step 5: Release the VideoWriter
out.release()